# <font color=black> Stock Data Ingestion </font>

> ## <font color=black> Imports: </font>

In [100]:
import requests
import pyodbc
import pandas as pd
import json
from sqlalchemy import create_engine
from sqlalchemy.engine import URL


> ## <font color=black> Build Connection String </font>


In [105]:
driver ='Driver={ODBC Driver 17 for SQL Server};'
server = 'SERVER=;'
database = 'DATABASE=;'
username = 'UID=;'
password = 'PWD='
cnxstr = driver + server + database + username + password
cnstr = "mssql+pyodbc://"
connurl = URL.create("mssql+pyodbc", query={"odbc_connect": cnxstr})



> ## <font color=black> IngestData </font>
> > ### <font color=black> This method takes in a ticker, and will <br> use the Yahoo Finance API to get the daily stock price for one year <br> I am using sqlalchemy, and pandas for the ELT pipeline </font>


In [138]:
def IngestData(ticker):
    url = "https://yahoofinance-stocks1.p.rapidapi.com/stock-prices"

    querystring = {"EndDateInclusive":"2022-06-28","StartDateInclusive":"2021-06-28","Symbol":ticker,"OrderBy":"Ascending"}

    headers = {
        "X-RapidAPI-Key": "",
        "X-RapidAPI-Host": ""
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    response_json = response.json()
    df = pd.DataFrame.from_dict(response_json['results'])
    df['ticker'] = ticker
    df.reset_index()
    engine = create_engine(connurl, fast_executemany=True)
    df.to_sql('StockData', schema="stonks", con=engine, if_exists='append' )
    


> ## <font color=black> Get Tickers </font>
> > ### <font color=black> I found a CSV online with the top 500 tickers <br> I I read the CSV into a pandas dataframe, and iterate through it, passing in each ticker to the data ingestion method </font>

In [141]:
df = pd.read_csv('toptickers.csv')

In [142]:
for row in df.itertuples(index=False):
    IngestData(row[0])
